In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bengkulu,ID,2021-05-22 03:32:45,-3.8004,102.2655,76.98,85,100,3.87
1,1,Hasaki,JP,2021-05-22 03:32:45,35.7333,140.8333,70.20,77,100,19.73
2,2,Narsaq,GL,2021-05-22 03:32:45,60.9167,-46.0500,34.02,85,97,5.64
3,3,Santa Fe,AR,2021-05-22 03:32:46,-31.6333,-60.7000,55.02,93,97,4.00
4,4,Busselton,AU,2021-05-22 03:32:46,-33.6500,115.3333,62.10,86,100,18.25


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
heat_layer = gmaps.heatmap_layer(locations, weights=temps)

In [8]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [11]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

In [13]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bengkulu,ID,2021-05-22 03:32:45,-3.8004,102.2655,76.98,85,100,3.87
6,6,Santa Marta,CO,2021-05-22 03:31:53,11.2408,-74.1990,86.05,83,75,6.85
11,11,Padang,ID,2021-05-22 03:30:40,-0.9492,100.3543,84.25,67,92,3.89
15,15,Wahiawa,US,2021-05-22 03:32:49,21.5028,-158.0236,79.12,61,1,19.57
16,16,Buchanan,LR,2021-05-22 03:32:49,5.8808,-10.0467,75.16,85,78,0.51
18,18,Kavaratti,IN,2021-05-22 03:32:50,10.5669,72.6420,82.56,74,93,5.61
21,21,Samarai,PG,2021-05-22 03:32:51,-10.6167,150.6667,80.20,81,85,20.13
25,25,Daru,PG,2021-05-22 03:32:52,-9.0763,143.2092,79.09,88,90,17.27
30,30,Atuona,PF,2021-05-22 03:32:54,-9.8000,-139.0333,80.04,71,21,18.23
31,31,Labuhan,ID,2021-05-22 03:32:54,-6.8844,112.2051,87.13,63,20,2.66


In [19]:
preferred_cities_df.count()

City_ID       168
City          168
Country       168
Date          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
dtype: int64

In [20]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [21]:
preferred_cities_df.notnull().sum()

City_ID       168
City          168
Country       168
Date          168
Lat           168
Lng           168
Max Temp      168
Humidity      168
Cloudiness    168
Wind Speed    168
dtype: int64

In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bengkulu,ID,76.98,-3.8004,102.2655,
6,Santa Marta,CO,86.05,11.2408,-74.1990,
11,Padang,ID,84.25,-0.9492,100.3543,
15,Wahiawa,US,79.12,21.5028,-158.0236,
16,Buchanan,LR,75.16,5.8808,-10.0467,
18,Kavaratti,IN,82.56,10.5669,72.6420,
21,Samarai,PG,80.20,-10.6167,150.6667,
25,Daru,PG,79.09,-9.0763,143.2092,
30,Atuona,PF,80.04,-9.8000,-139.0333,
31,Labuhan,ID,87.13,-6.8844,112.2051,


In [23]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [25]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
  #  hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bengkulu,ID,76.98,-3.8004,102.2655,Grage Hotel Bengkulu
6,Santa Marta,CO,86.05,11.2408,-74.1990,Casa de Isabella
11,Padang,ID,84.25,-0.9492,100.3543,Mercure Padang
15,Wahiawa,US,79.12,21.5028,-158.0236,Aloha Wahiawa
16,Buchanan,LR,75.16,5.8808,-10.0467,Kamaneahn Hotel Inc
18,Kavaratti,IN,82.56,10.5669,72.6420,Hotels in Lakshadweep Islands
21,Samarai,PG,80.20,-10.6167,150.6667,Nuli Sapi
25,Daru,PG,79.09,-9.0763,143.2092,Daru Lodge
30,Atuona,PF,80.04,-9.8000,-139.0333,Villa Enata
31,Labuhan,ID,87.13,-6.8844,112.2051,PT PPI Lamongan Tambak C


In [27]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [30]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))